In [1]:
# !wget https://github.com/stefan-it/nmt-en-vi/raw/master/data/train-en-vi.tgz
# !tar -zxf train-en-vi.tgz
# !wget https://github.com/stefan-it/nmt-en-vi/raw/master/data/dev-2012-en-vi.tgz
# !tar -zxf dev-2012-en-vi.tgz
# !wget https://github.com/stefan-it/nmt-en-vi/raw/master/data/test-2013-en-vi.tgz
# !tar -zxf test-2013-en-vi.tgz

In [2]:
# !pip3 install malaya --no-deps
# !pip3 install bert-tensorflow
# !pip3 install toolz
# !pip3 install pysastrawi
# !pip3 install fuzzywuzzy
# !pip3 install xgboost
# !pip3 install ftfy

In [3]:
import malaya
import re

tokenizer = malaya.preprocessing.SocialTokenizer().tokenize

def is_number_regex(s):
    if re.match("^\d+?\.\d+?$", s) is None:
        return s.isdigit()
    return True

def preprocessing(string):
    tokenized = tokenizer(string)
    tokenized = ['<NUM>' if is_number_regex(w) else w for w in tokenized]
    return tokenized

In [4]:
with open('train.en') as fopen:
    train_english = fopen.read().split('\n')[:-1]
    
with open('train.vi') as fopen:
    train_vietnam = fopen.read().split('\n')[:-1]
    
len(train_english), len(train_vietnam)

(133317, 133317)

In [5]:
from tqdm import tqdm

for i in tqdm(range(len(train_english))):
    train_english[i] = ' '.join(preprocessing(train_english[i]))

100%|██████████| 133317/133317 [00:16<00:00, 8000.84it/s]


In [6]:
for i in tqdm(range(len(train_vietnam))):
    train_vietnam[i] = ' '.join(preprocessing(train_vietnam[i]))

100%|██████████| 133317/133317 [00:22<00:00, 6040.29it/s]


In [7]:
with open('tst2012.en') as fopen:
    test_english_2012 = fopen.read().split('\n')[:-1]
    
with open('tst2012.vi') as fopen:
    test_vietnam_2012 = fopen.read().split('\n')[:-1]
    
len(test_english_2012), len(test_vietnam_2012)

(1553, 1553)

In [8]:
for i in tqdm(range(len(test_english_2012))):
    test_english_2012[i] = ' '.join(preprocessing(test_english_2012[i]))
    
for i in tqdm(range(len(test_vietnam_2012))):
    test_vietnam_2012[i] = ' '.join(preprocessing(test_vietnam_2012[i]))

100%|██████████| 1553/1553 [00:00<00:00, 6857.35it/s]


In [9]:
with open('tst2013.en') as fopen:
    test_english_2013 = fopen.read().split('\n')[:-1]
    
with open('tst2013.vi') as fopen:
    test_vietnam_2013 = fopen.read().split('\n')[:-1]
    
len(test_english_2013), len(test_vietnam_2013)

(1268, 1268)

In [10]:
for i in tqdm(range(len(test_english_2013))):
    test_english_2013[i] = ' '.join(preprocessing(test_english_2013[i]))
    
for i in tqdm(range(len(test_vietnam_2013))):
    test_vietnam_2013[i] = ' '.join(preprocessing(test_vietnam_2013[i]))

100%|██████████| 1268/1268 [00:00<00:00, 5696.79it/s]


In [11]:
train_X, train_Y = [], []
for i in range(len(train_english)):
    if len(train_english[i].split()) > 250:
        continue
    train_X.append(train_english[i])
    train_Y.append(train_vietnam[i])

In [12]:
test_X, test_Y = [], []
for i in range(len(test_english_2012)):
    if len(test_english_2012[i].split()) > 250:
        continue
    test_X.append(test_english_2012[i])
    test_Y.append(test_vietnam_2012[i])
    
for i in range(len(test_english_2013)):
    if len(test_english_2013[i].split()) > 250:
        continue
    test_X.append(test_english_2013[i])
    test_Y.append(test_vietnam_2013[i])

In [13]:
len(train_X) + len(test_X)

136109

In [19]:
test_Y[:5]

['Làm sao tôi có thể trình bày trong <NUM> phút về sợi dây liên kết những người phụ nữ qua ba thế hệ , về việc làm thế nào những sợi dây mạnh mẽ đáng kinh ngạc ấy đã níu chặt lấy cuộc sống của một cô bé bốn tuổi co quắp với đứa em gái nhỏ của cô bé , với mẹ và bà trong suốt năm ngày đêm trên con thuyền nhỏ lênh đênh trên Biển Đông hơn <NUM> năm trước , những sợi dây liên kết đã níu lấy cuộc đời cô bé ấy và không bao giờ rời đi - - cô bé ấy giờ sống ở San Francisco và đang nói chuyện với các bạn hôm nay ?',
 'Câu chuyện này chưa kết thúc .',
 'Nó là một trò chơi ghép hình vẫn đang được xếp .',
 'Hãy để tôi kể cho các bạn về vài mảnh ghép nhé .',
 'Hãy tưởng tượng mảnh đầu tiên : một người đàn ông đốt cháy sự nghiệp cả đời mình .']

In [16]:
import youtokentome as yttm

In [21]:
with open('text.txt', 'w') as fopen:
    fopen.write('\n'.join(train_X))
    
english = yttm.BPE.train(data='text.txt', vocab_size=32000, model='english.model',
              pad_id=0, unk_id=2, bos_id=3, eos_id=1)

In [22]:
with open('text.txt', 'w') as fopen:
    fopen.write('\n'.join(train_Y))
    
vietnam = yttm.BPE.train(data='text.txt', vocab_size=32000, model='vietnam.model',
              pad_id=0, unk_id=2, bos_id=3, eos_id=1)

In [28]:
train_X = english.encode(train_X, output_type=yttm.OutputType.ID)
train_X = [i + [1] for i in train_X]
test_X = english.encode(test_X, output_type=yttm.OutputType.ID)
test_X = [i + [1] for i in test_X]

In [29]:
train_Y = vietnam.encode(train_Y, output_type=yttm.OutputType.ID)
test_Y = vietnam.encode(test_Y, output_type=yttm.OutputType.ID)

In [31]:
import json

with open('train-test-bpe.json', 'w') as fopen:
    json.dump({'train_X': train_X, 'train_Y': train_Y,
              'test_X': test_X,
              'test_Y': test_Y}, fopen)